In [ ]:
# this code is intended to be a minimum viable reproduction of the process for adding a bus stop in Genet to try and understand why it is throwing a mobsim error.

from genet import Route, Service, Stop, read_matsim
import pandas as pd
import os

# baseline 2040 network
path_to_matsim_network = "/mnt/c/_BERTIE_data/2_2040_do_minimum_40WFH/"
# chosen output location
output_path = "/mnt/c/_BERTIE_data/bug_test"


/home/siskiyoucedar/micromamba/envs/genet/lib/python3.12/site-packages/keplergl/keplergl.py:2: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_string


In [ ]:
# read the network:

network = os.path.join(path_to_matsim_network, "output_network.xml")
schedule = os.path.join(path_to_matsim_network, "output_transitSchedule.xml") 
vehicles = os.path.join(path_to_matsim_network, 'output_transitVehicles.xml')
n = read_matsim(
    path_to_network=network, epsg="epsg:27700", path_to_schedule=schedule, path_to_vehicles=vehicles
)

In [ ]:
# check the network:

n

<Network instance at 135886629621584: with 
graph: MultiDiGraph with 447655 nodes and 985382 edges and 
schedule Schedule:
Number of services: 1351
Number of routes: 4687
Number of stops: 20219

In [4]:
# rep = n.generate_validation_report() 

# just checking everything is okay before we take things any further - if there is a speed error, then we know we cannot trust the validation reports!

# ok, no speed error - so it's definitely something we did.

In [ ]:
# bus stop IDs have changed since older renditions of BERTIE
# fixed it. was 5177495089997857897; dug out relevant stop id from schedule xml
# appears stop ids have changed in the newer versions of BERTIE

n.schedule.stop('390060143').print()

Stop ID: 390060143
Projection: epsg:27700
Lat, Lon: 52.150799, 1.60107101
linkRefId: 5177495089997857897_5177495089997857897


In [ ]:
# two relevant stops for MVP - they are quite far away, sorry!

market_town_stops = {
    'Aldeburgh': '390060144',
    'Woodbridge': '390060607'
}

In [ ]:
# adding a new route:

route = Route(
    route_short_name="new_route",
    mode="bus",
    # five services between 7am and 8am
    headway_spec={("07:00:00", "08:00:00"): 15},
    arrival_offsets=["00:00:00","00:04:00",],
    departure_offsets=["00:00:00","00:04:00",],
    id="new_route",
    # do not need exact route; route using shutil later.
    #    route= ['834', '1573', '3139', '3141', '574', '3154', '979', '980', '981'],
    await_departure=[True, True],
    # two stops, in woodbridge and aldeburgh
    stops=[
        n.schedule.stop("390060144"),
        n.schedule.stop("390060607"),
    ],
)

# vehs created automatically, see below

In [ ]:
# check everything looks normal:

# route.trips

{'trip_id': ['new_route_07:00:00',
  'new_route_07:15:00',
  'new_route_07:30:00',
  'new_route_07:45:00',
  'new_route_08:00:00'],
 'trip_departure_time': ['07:00:00',
  '07:15:00',
  '07:30:00',
  '07:45:00',
  '08:00:00'],
 'vehicle_id': ['veh_bus_new_route_07:00:00',
  'veh_bus_new_route_07:15:00',
  'veh_bus_new_route_07:30:00',
  'veh_bus_new_route_07:45:00',
  'veh_bus_new_route_08:00:00']}

In [ ]:
# turn route into service:

new_service = Service(id="new_service", routes=[route])
new_service.print()

Service ID: new_service
Name: new_route
Number of routes: 1
Number of stops: 2


In [ ]:
# check the number of vehicles in the schedule:

len(n.schedule.vehicles)

In [ ]:
# add the service to the schedule:

n.schedule.add_service(Service(id="new_service", routes=[route]))
n.schedule.generate_vehicles()

In [ ]:
# check the number of vehicles in the schedule:

len(n.schedule.vehicles)

# good, so five instances of that service were added!

In [ ]:
# from genet reference docs - says no problem        

n.schedule.validate_vehicle_definitions()

In [ ]:
# check schedule has been changed correctly.

n.schedule.change_log().tail()

# ok, it's in the frame, just have to hope it runs properly in the model...

# may need to add 'force=True' to end of changes at schedule.add_route - but not convinced this is the error at the moment.

In [ ]:
# this output is also correct

n.schedule.vehicle_types

In [ ]:
# this should be empty - in other words, the new service has yet to be connected to the network:

n.schedule["new_service"].route("new_route").network_links

In [ ]:
# make sure stops are ordered:

n.schedule["new_service"].route("new_route").ordered_stops

In [ ]:
# now that the service is in the schedule - we can route it:

import shutil
# I installed CBC to make this work: https://github.com/coin-or/Cbc

if shutil.which("cbc"):
    n.route_service("new_service")
else:
    print("Cannot route service without a solver installed")

In [ ]:
# this now yields a long list - showing that yes, the bus route has been routed

n.schedule["new_service"].route("new_route").network_links

In [ ]:
# this is the same output but still looks good

n.schedule["new_service"].route("new_route").ordered_stops

In [ ]:
# vehicle definition check - from DeepWiki:

print(f"Total vehicles: {len(n.schedule.vehicles)}")  
print(f"Vehicle types: {n.schedule.vehicle_types.keys()}")  
  
print(new_service.routes)

# Check vehicles for your new service specifically  
# new_service = n.schedule["new_service"]  
# for r in new_service.routes:  
#     print(f"Route {route.id} vehicles: {r.vehicles}")  
  
# # Validate vehicle definitions  
n.schedule.validate_vehicle_definitions()  
missing_info = n.schedule.get_missing_vehicle_information()  
print(f"Missing vehicle info: {missing_info}")

In [ ]:
# service routing check - from DeepWiki:

# Check if the service was properly routed  
route = n.schedule["new_service"].route("new_route")  
print(f"Network links: {len(route.network_links) if route.network_links else 0}")  
print(f"Ordered stops: {route.ordered_stops}")  
  
# Verify all stops have link references  
for stop in route.stops():  
    has_link = hasattr(stop, 'linkRefId')  
    print(f"Stop {stop.id}: has linkRefId = {has_link}")

In [ ]:
# verify stop facilities - from DeepWiki:

for stop_id in ["390060144", "390060607"]:  
    stop = n.schedule.stop(stop_id)  
    print(f"Stop {stop_id}:")  
    print(f"  ID: {stop.id}")  
    print(f"  Link ref: {getattr(stop, 'linkRefId', 'MISSING')}")  
    print(f"  Attributes: {getattr(stop, 'attributes', {})}")

In [ ]:
# check structural questions - from DeepWiki:

# Check for common structural issues  
print(f"Services: {n.schedule.service_ids()}")  
print(f"Routes per service: {n.schedule.service_to_route_map()}")  
  
# Ensure your new service is properly structured  
new_service = n.schedule["new_service"]  
print(f"New service routes: {new_service.route_ids()}")  

for route_id in new_service.route_ids():  
    route = new_service.route(route_id)  
    print(f"Route {route_id}: {len(list(route.stops()))} stops, {len(route.vehicles())} vehicles")

In [ ]:
# Check the raw graph structure - from DeepWiki:
print(f"Number of routes: {len(n.schedule.route_ids())}")  
print(f"Number of services: {len(n.schedule.service_ids())}")  

In [ ]:
# Check each route's data consistency - from DeepWiki: 
for route_id in n.schedule.route_ids():  
    route_data = n.schedule._graph.graph["routes"][route_id]  
    print(f"Route {route_id}:")  
    print(f"  Ordered stops: {len(route_data.get('ordered_stops', []))}")  
    print(f"  Arrival offsets: {len(route_data.get('arrival_offsets', []))}")  
    print(f"  Departure offsets: {len(route_data.get('departure_offsets', []))}")  
    print(f"  Trip IDs: {len(route_data.get('trips', {}).get('trip_id', []))}")  
#     # Check for mismatched lengths  
    stops = route_data.get('ordered_stops', [])  
    arrivals = route_data.get('arrival_offsets', [])  
    departures = route_data.get('departure_offsets', [])  
    trips = route_data.get('trips', {}).get('trip_id', [])  
      
    if len(stops) - 1 != len(arrivals):  
        print(f"  ERROR: Stop-arrival mismatch: {len(stops)-1} vs {len(arrivals)}")  
    if len(stops) - 1 != len(departures):  
        print(f"  ERROR: Stop-departure mismatch: {len(stops)-1} vs {len(departures)}")

In [ ]:
# check if route specifically is valid - it says it is

route = new_service.route("new_route")
is_valid, invalid_stages = route.is_valid_route(return_reason=True)
print(f"Valid: {is_valid}, Failed stages: {invalid_stages}")

In [ ]:
# full validation report - not recommended as takes a honking long time

rep = n.generate_validation_report()

In [ ]:
# further genet checks via DeepWiki:

# # Check if all stops have consistent coordinate projections  
# has_uniform_projection = n.schedule.has_uniformly_projected_stops()  
# print(f"Uniform projections: {has_uniform_projection}")  
  
# # Get unique projections to identify inconsistencies  
# unique_projections = n.schedule.unique_stop_projections()  
# print(f"Unique projections: {unique_projections}")  
  
# # Check for conflicting stop data  
# stops_without_data, stops_with_conflicting_data = n.schedule._compare_stops_data(graph_reference)  
# print(f"Stops without data: {stops_without_data}")  
# print(f"Stops with conflicting data: {stops_with_conflicting_data}")

In [ ]:
# reindexing - not currently necessary, but available: 

# n.schedule["new_service"].reindex(new_id="more_appropriate_id")

In [ ]:
# provided the above suggests new_service is valid, write to disk.

n.write_to_matsim(output_path)